<a href="https://colab.research.google.com/github/ryam14/Portfolio_website/blob/main/whisper.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
from IPython.display import HTML
from base64 import b64decode

In [ ]:
RECORD = """
<!DOCTYPE html>
<html lang="en">
<head>
  <meta charset="UTF-8">
  <meta name="viewport" content="width=device-width, initial-scale=1.0">
  <title>Audio Recorder</title>
</head>
<body>
  <h1>Audio Recorder</h1>
  <button id="startButton">Start</button>
  <button id="endButton">End</button>
  <p id="log"></p>

  <script>
    const sleep = time => new Promise(resolve => setTimeout(resolve, time));
    const b2text = blob => new Promise(resolve => {
      const reader = new FileReader();
      reader.onloadend = e => resolve(e.srcElement.result);
      reader.readAsDataURL(blob);
    });

    let recorder = null;
    let chunks = [];
    let stream = null;

    const record = () => new Promise((resolve, reject) => {
      const startRecording = async () => {
        try {
          stream = await navigator.mediaDevices.getUserMedia({ audio: true });
          recorder = new MediaRecorder(stream);
          chunks = [];

          recorder.ondataavailable = e => chunks.push(e.data);

          recorder.start();
          document.getElementById("log").innerText = "Recording started...";
        } catch (err) {
          console.error("Error accessing microphone: ", err);
          reject(err);
        }
      };

      const stopRecording = async () => {
        if (!recorder || recorder.state !== "recording") {
          document.getElementById("log").innerText = "No active recording to stop.";
          reject("No active recording");
          return;
        }

        recorder.onstop = async () => {
          const blob = new Blob(chunks);
          const text = await b2text(blob);
          console.log("Recording stopped");
          document.getElementById("log").innerText = "Recording stopped. Check console for Base64 audio data.";
          resolve(text);
          stream.getTracks().forEach(track => track.stop());
        };

        recorder.stop();
      };

      document.getElementById("startButton").addEventListener("click", startRecording);
      document.getElementById("endButton").addEventListener("click", stopRecording);
    });
  </script>
</body>
</html>
"""

def record():
  try:
    from google.colab import output
  except ImportError:
    print('No possible to import output from google.colab')
    return ''
  else:
    print('Recording')
    # Display HTML interface
    display(HTML(RECORD))

    # Use JavaScript to start and stop recording, and return Base64 audio data
    try:
      s = output.eval_js('record()')  # This will return Base64-encoded audio data
      fname = '/content/recorded_audio.wav'
      print('Saving to', fname)

      # Decode Base64 and save as a .wav file
      b = b64decode(s.split(',')[1])
      with open(fname, 'wb') as f:
        f.write(b)
      return fname
    except Exception as e:
      print(f"An error occurred: {e}")
      return ''

In [ ]:
!pip install git+https://github.com/openai/whisper.git


  Cloning https://github.com/openai/whisper.git to /tmp/pip-req-build-7dzg18p_
  Running command git clone --filter=blob:none --quiet https://github.com/openai/whisper.git /tmp/pip-req-build-7dzg18p_
  Resolved https://github.com/openai/whisper.git to commit 90db0de1896c23cbfaf0c58bc2d30665f709f170
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done


In [ ]:
import whisper

In [ ]:
model = whisper.load_model('medium')

100%|█████████████████████████████████████| 1.42G/1.42G [00:19<00:00, 77.5MiB/s]
/usr/local/lib/python3.10/dist-packages/whisper/__init__.py:150: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this exper

In [ ]:
record()

Recording


Saving to /content/recorded_audio.wav


'/content/recorded_audio.wav'

In [ ]:
result = model.transcribe('/content/recorded_audio.wav', verbose=True)
print(result['text'])

with open("/content/transcription.txt", "a", encoding="utf-8") as txt:
    txt.write(result["text"])

Detecting language using up to the first 30 seconds. Use `--language` to specify the language


/usr/local/lib/python3.10/dist-packages/whisper/transcribe.py:132: UserWarning: FP16 is not supported on CPU; using FP32 instead
  warnings.warn("FP16 is not supported on CPU; using FP32 instead")


Detected language: Japanese
[00:00.000 --> 00:13.000] 視聴率はもしの結果、今でしょの名フレーズが、YOU CANの信号流行語大賞に選ばれてからすでに11年が経った。
視聴率はもしの結果、今でしょの名フレーズが、YOU CANの信号流行語大賞に選ばれてからすでに11年が経った。
